In [4]:
import pandas as pd
import os

In [5]:
quarters = [
    "2024-09-30", "2024-06-30", "2024-03-31", "2023-12-31", "2023-09-30",
    "2023-06-30", "2023-03-31", "2022-12-31", "2022-09-30", "2022-06-30",
    "2022-03-31", "2021-12-31", "2021-09-30", "2021-06-30", "2021-03-31",
    "2020-12-31", "2020-09-30", "2020-06-30", "2020-03-31"
]
QnY = ['Q3 2024', 'Q2 2024', 'Q1 2024', 'Q4 2023', 'Q3 2023',
 'Q2 2023', 'Q1 2023', 'Q4 2022', 'Q3 2022', 'Q2 2022',
 'Q1 2022', 'Q4 2021', 'Q3 2021', 'Q2 2021', 'Q1 2021',
 'Q4 2020', 'Q3 2020', 'Q2 2020', 'Q1 2020']

icm_index = ['Revenue', 'Net Income', 'EPS (Diluted)', 'Dividend Per Share']
blc_index = ['Total Assets', 'Total Liabilities', "Shareholders' Equity", 'Book Value Per Share']
ratio_index = [ 
    'Market Capitalization',
    'PE Ratio', 'PS Ratio', 'PB Ratio', 'Return on Assets (ROA)', 'Return on Equity (ROE)',
    'Earnings Yield', 'Dividend Yield'
]
exchanges = ['SHA', 'SHE', 'HOSE','HNX']

#### Clean the raw data

In [6]:
data = pd.DataFrame()
ex = "HKG"
comp_df = pd.read_csv(f'./exchange/{ex}/{ex}.csv')
count = 0
comp_list = comp_df["s"]
for comp in comp_list:
    try:
        comp = comp.split('/')[1]
        icm_stm_df = pd.read_csv(f'./exchange/{ex}/{comp}/income.csv',header=[0], index_col=1)
        icm_stm_df = icm_stm_df.rename(index={"Total Revenue": "Revenue"})
        balance_df = pd.read_csv(f'./exchange/{ex}/{comp}/balance.csv',header=[0], index_col=1)
        ratio_df = pd.read_csv(f'./exchange/{ex}/{comp}/ratio.csv',header=[0], index_col=1)
        price_df = pd.read_csv(f'./exchange/{ex}/{comp}/price.csv',header=[0])
        price_df["Date"] = pd.to_datetime(price_df["Date"])
        price_df["Quarters"] = "Q" + price_df["Date"].dt.quarter.astype(str) + " " + price_df["Date"].dt.year.astype(str)
        
        price_df = price_df.set_index("Quarters")
        
        icm_stm_df.drop("Period Ending", axis = 0, inplace= True)
        icm_stm_df.drop("Unnamed: 0", axis = 1, inplace= True)
        icm_stm_df = icm_stm_df.filter(items = QnY)

        balance_df.drop("Period Ending", axis = 0, inplace= True)
        balance_df.drop("Unnamed: 0", axis = 1, inplace= True)
        balance_df = balance_df.filter(items = QnY)

        ratio_df.drop("Period Ending", axis = 0, inplace= True)
        ratio_df.drop("Unnamed: 0", axis = 1, inplace= True)
        ratio_df = ratio_df.filter(items = QnY)
        
        try:
            icm_stm_df = icm_stm_df.loc[icm_index]      
            balance_df = balance_df.loc[blc_index]
            ratio_df = ratio_df.loc[ratio_index]
            price_df = price_df.T

            # print(price_df.loc["Stock Price"])

            combined_df = pd.concat([icm_stm_df,balance_df,ratio_df,price_df], axis=0).T
            # print(combined_df)
            combined_df["Symbol"] = comp
            combined_df.columns.name = None
            data = pd.concat([data,combined_df], axis= 0)
            print(data.index[data.index.duplicated()])
            print(data.columns[data.columns.duplicated()])
   
            print(data)
            
        except Exception:    
            continue
        # output_dir = f"./cleaned/{ex}/{comp}"
        # os.makedirs(output_dir, exist_ok=True)
        # icm_stm_df.to_csv(output_dir,header=True, index_label="Fiscal Quarter")
        
    except FileNotFoundError as e:
        continue
data.to_csv("shenzhen_2.csv")
print(count)


Index([], dtype='object')
Index([], dtype='object')
        Revenue Net Income EPS (Diluted) Dividend Per Share Total Assets  \
Q3 2024  167193      53230          5.64                  -      1723357   
Q2 2024  161117      47630          4.99                  -      1654970   
Q1 2024  159501      41889          4.39                  -      1646677   
Q4 2023  155196      27025          2.79              3.086      1577246   
Q3 2023  154625      36182          3.75                  -      1541233   
Q2 2023  149208      26171          2.69                  -      1537503   
Q1 2023  149986      25838          2.64                  -      1517206   
Q4 2022  144954     106268         10.95              2.123      1578131   
Q3 2022  140093      39943          4.10                  -      1478143   
Q2 2022  134034      18619          1.92                  -      1459669   
Q1 2022  135471      23413          2.40                  -      1494158   
Q4 2021  144188      94958          